<a href="https://colab.research.google.com/github/bestKUFO/ApiCrawling/blob/main/%EC%82%AC%EC%9D%B4%EC%96%B8%EC%8A%A4%EC%98%A8%20%EB%85%BC%EB%AC%B8%20api%EA%B5%AC%ED%98%84_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

git에서 Khaiii호출 및 pip 인스톨

In [ ]:
!git clone https://github.com/kakao/khaiii.git

!mkdir build

!cd build && cmake /content/khaiii

!cd /content/build/ && make all

!cd /content/build/ && make resource

!cd /content/build && make install

!cd /content/build && make package_python

!pip install /content/build/package_python

패키지 설치

In [ ]:
!pip install --upgrade gensim
!pip install konlpy

URL 호출

In [ ]:
import requests
import re
import pandas as pd

#URL 작동 형식 => Baseurl + Access_Token + Version + Action + Target + SearchQuery + SortField + CurPage + str(page) + RowCount
Baseurl = "https://apigateway.kisti.re.kr/openapicall.do?client_id={Client_id를 입력하세요}}"
Access_Token = "&token= #Access_Token을 입력하면 됩니다." # 2시간마다 갱신되는 토큰
Version = "&version=1.0"
Action = "&action=search" # API액션 구분
Target = "&target=ARTI" #논문 약어
field = "{\"BI\":"
search = "\"메타버스\"}"
SearchQuery = "&searchQuery="+field+search # 검색필드 항목 : 검색어
SortField = "&sortField=title" #정렬 필드
CurPage = "&curPage=" #현재 페이지, cur*row = 10,000건 미만으로 제한
row_count = 10 #한 페이지당 출력 건수
RowCount = "&rowCount="+str(row_count) #디스플레이 건수 기본값:10, 최대값: 100

row_num = []
article_title = []
article_abstract = []

논문 내용 호출 및 저장

In [ ]:
#정해진 페이지 탐구하기
class Article:
  def __init__(self, max_page_num):
    self.max_page_num = max_page_num

  def getURL(self): # url을 만든 후 텍스트화 시킨 뒤 논문명과 초록을 각각 추출해 리스트에 저장하는 것을 반복
    page = 1
    
    for page in range(1,self.max_page_num+1):
      url = Baseurl + Access_Token + Version + Action + Target + SearchQuery + SortField + CurPage + str(page) + RowCount

      response = requests.get(url)  
      page_text = response.text
      

      for r in range(1,row_count+1):
        #논문 번호 추가
        page_row_num = f"{page}_{r}"
        row_num.append(page_row_num)

        #타이틀 호출 및 리스트에 저장
        startidx = page_text.find("논문명")
        page_text = page_text[startidx+14:]
        lastidx = page_text.find('</item>')
        article_title.append(page_text[:lastidx-3])

        #초록 호출 및 리스트에 저장
        startidx = page_text.find("초록")
        page_text = page_text[startidx+13:]
        lastidx = page_text.find('</item>')
        article_abstract.append(page_text[:lastidx-3])
      
      page += 1



  def callAll(self): # 리스트에 저장한 모든 문서 불러옴
    for a in range(len(row_num)):
      print(row_num[a], article_title[a], article_abstract[a], sep="\n")

  def analysis_title(self):
    #논문 제목 분석
    str_title = " ".join(article_title) # 문자열로 변환
    str_title_upper = str_title.upper() # 대문자로 변환
    str_title_upper = re.sub("[《》:<>‘’$/^]"," ",str_title_upper) #특수기호 제거
    split_title = str_title_upper.split() # split으로 단어로 나눠줌
    title_count={}
    for i in split_title:
      try: title_count[i] += 1 
      except: title_count[i] = 1
    
    print(title_count)
  
  def analysis_abstract(self):
    #논문 초록 분석
    str_abstract = " ".join(article_abstract)
    str_abstract_upper1 = str_abstract.upper()
    str_abstract_upper = re.sub("[《》:<>‘’$/^]"," ",str_abstract_upper1)
    split_abstract = str_abstract_upper.split()
    select_abstract = []
    abstract_count={}
    for i in split_abstract:
      try: abstract_count[i] += 1
      except: abstract_count[i] = 1    
    
    print(abstract_count)

In [ ]:
#논문 검색 최대 페이지(페이지*페이지당 검색 갯수 <= 10,000 ), 한 페이지당 검색 갯수(기본값 10, 최대 100)
art = Article(30)
art.getURL()

#중복값 제거
# set_article_title = set(article_title)
# article_title = list(set_article_title)
# set_article_abstract = set(article_abstract)
# article_abstract = list(set_article_abstract)

#논문 타이틀 및 초록 갯수 확인
print(len(article_title))
print(len(article_abstract))

300
300


논문을 csv파일로 저장

In [ ]:
arti_df = pd.DataFrame({"Num":row_num,"Title":article_title,"Abstract":article_abstract})

arti_df.to_csv("논문.csv", encoding="utf-8-sig")

데이터를 불러와 khaiii로 형태소 분석

명사만 추출하는 버전

In [ ]:
from khaiii import KhaiiiApi
import numpy as np

api = KhaiiiApi()

Noun_list = []

article_abstract = list(filter(None, article_abstract))  # 공백 요소 제거 / 제거 안할 시 khaiii에서 오류남

data = " ".join(article_abstract)

for word in api.analyze(data):
  for morph in word.morphs:
    if morph.tag in ['NNG', 'NNP']:
      Noun_list.append(morph.lex)


#명사 리스트 갯수 확인
print(len(Noun_list))


16465


기호, 영어제외 추출 버전

In [ ]:
from khaiii import KhaiiiApi
import numpy as np
import re

api = KhaiiiApi()

article_abstract = list(filter(None, article_abstract))  # 공백 요소 제거 / 제거 안할 시 khaiii에서 오류남

re_abstract = []

for i in article_abstract:
  a = re.sub("&amp;#xD;"," ",i)
  b = re.sub("[<TEX$^/>]","",a)
  for b in api.analyze(b):
    for morph in b.morphs:
      if morph.tag not in ['SF', 'SP', 'SS', 'SE', 'SO', 'SW','SL','SN']:
        re_abstract.append(morph.lex)

#리스트 갯수 확인
print(len(re_abstract))

41001


Gensim을 이용한 형태소 분석

토큰화, 단어 학습

In [ ]:
from konlpy.tag import Okt
import matplotlib.pyplot as plt
from tqdm import tqdm

okt = Okt()

tokenized_data = []
for sentence in tqdm(Noun_list): # tqdm은 시각화용, 제외해도 무관
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    tokenized_data.append(tokenized_sentence)

100%|██████████| 16465/16465 [00:13<00:00, 1232.36it/s]


In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors

#size = 임배딩 백터차원 / window = 컨텍스트윈도우 크기 / min_count = 단어 최소 빈도 수 제한
#workers = 학습을 위한 프로세스 수 / 0 = CBOW / 1 = Skip - gram
model = Word2Vec(sentences=tokenized_data, vector_size=300, window=2, min_count=2, workers=4, sg=1)
model.wv.vectors.shape

(1561, 300)

Gensim을 이용한 단어 연관분석

In [ ]:
find_relation_word = input("")

model_result = model.wv.most_similar(find_relation_word)
print(model_result)

메타
[('량', 0.17684349417686462), ('상황', 0.16628172993659973), ('수용', 0.1647205948829651), ('도구', 0.15885388851165771), ('상위', 0.15422379970550537), ('국내', 0.1530960202217102), ('촬영', 0.15301477909088135), ('제작자', 0.15188434720039368), ('신설', 0.15149840712547302), ('수단', 0.14905992150306702)]
